# Setup

In [6]:
import helper_tools.parser as parser
import importlib
import pandas as pd

importlib.reload(parser)

relation_df, entity_df, docs = parser.synthie_parser("train")

Fetching 27 files:   0%|          | 0/27 [00:00<?, ?it/s]

100%|██████████| 10/10 [00:00<00:00, 6989.34it/s]


Uploading Entities to Qdrant.


100%|██████████| 46/46 [00:07<00:00,  6.28it/s]


Uploading Predicates to Qdrant.


100%|██████████| 29/29 [00:03<00:00,  8.20it/s]


In [7]:
from langchain_openai import ChatOpenAI
from langchain_ollama.embeddings import OllamaEmbeddings
from langfuse.callback import CallbackHandler
from dotenv import load_dotenv
import os

load_dotenv()
langfuse_handler = CallbackHandler(
    secret_key=os.getenv("LANGFUSE_SECRET_KEY"),
    public_key=os.getenv("LANGFUSE_PUBLIC_KEY"),
    host=os.getenv("LANGFUSE_HOST"),
)

model = ChatOpenAI(model_name="Meta-Llama-3.3-70B-Instruct", base_url="https://api.sambanova.ai/v1", api_key=os.getenv("SAMBANOVA_API_KEY"))
embeddings = OllamaEmbeddings(model='nomic-embed-text')

In [8]:
target_doc = docs.iloc[0]
doc_id = target_doc["docid"]
text = target_doc["text"]
text

'Corfe Castle railway station is a station on the Swanage Railway in the village of Corfe Castle, in the United Kingdom.'

In [9]:
from langchain_qdrant import QdrantVectorStore
from qdrant_client import QdrantClient

client = QdrantClient("localhost", port=6333)
vector_store = QdrantVectorStore(
    client=client,
    collection_name="wikidata_labels",
    embedding=embeddings
)

In [13]:
from langgraph.types import Command
from typing import TypedDict, Literal
from langchain_core.prompts import PromptTemplate
import re
from langchain_core.messages import AIMessage
from langgraph.graph import StateGraph, MessagesState, START, END

class cIEState(TypedDict):
    text: str
    call_trace: list[tuple[str]]
    results: list[str]
    comments: list[AIMessage]
    instruction: str
    
def planner(state: cIEState) -> Command[Literal["agent_instructor_agent"]]:
    prompt = PromptTemplate.from_template("""
    You are an expert in planning and executing tasks within multi-agent systems. Your role is to design and refine a detailed plan that processes a given text into a triple format, specifically for closed information extraction using an underlying Knowledge Graph. You design the plan for the agent instructor agent, which should execute your plan, call and instruct agents. It is only able to execute one step at a time. Your plan must be based on the following inputs:
    - Agent Call Trace
    - Agent Comments
    - The provided input text
    - All intermediate results produced during the process
    
    For executing the tasks, you can include the following agents in the plan:
    - **Entity Extraction Agent:** Can extract entities from the text.
    - **Relation Extraction Agent:** Can extract relations from the text.
    - **URI Detection Agent:** Based on search terms, can determine if there is an associated entity or relation in the Knowledge Graph.
    
    Your plan should clearly outline the steps required to achieve the goal, ensuring that each phase is actionable and verifiable. The plan will be passed to the Agent Instructor, who will execute the steps through a series of Agent Calls. You will be asked to build up a plan, as long as no final result is done. Your response should be precise, structured, and demonstrate deep expertise in orchestrating complex multi-agent systems for closed Information Extraction tasks. Please line up the plan that you have, to accomplish the task. Do not include tasks that are already worked on. Your plan does not have to include steps like triple formation or verification as this is either covered by the result checker agent or externally.
    
    If you are called for the first time write down the full plan. If you are called afterwards just say what the next task is and where in your plan we are.
    
    Please base your plan on the following information:
    
    Agent Call Trace: {call_trace}
    Agent Comments: {comments}
    The provided input text: {text}
    All intermediate results produced during the process: {results}
    """)
   
    response_chain = prompt | model
    
    response = response_chain.invoke(state)
          
    return Command(goto="agent_instructor_agent", update={"comments": state["comments"] + [response]})

def agent_instructor_agent(state: cIEState) -> Command[Literal["entity_extraction_agent", "relation_extraction_agent", "uri_detection_agent"]]:
    prompt = PromptTemplate.from_template("""
    
     You are an expert for executing plans in multi-agent-systems and instructing agents. You are embedded within such a MAS with the final goal of processing a text into relations. You will receive a plan from a planning agent within the agent comments alongside with the feedback given by the result checker. In addition, you will receive your agent call traces and the text which is being processed. Your task is then to reason, how the next agent should be called. The planner might give you a hint, which agent should be called next. 
     
    You have access on the following agents:
    Entity Extraction Agent
    - id: entity_extraction_agent
    - use of instruction: The use of an instruction is optional. It will be included in the context of the prompt of the agent and can modify the agents behaviour. Please do not include the original text in the prompt.
    - description: Can extract entities from the text.
    - state access on: text, instruction
    
    Relation Extraction Agent
    - id: relation_extraction_agent
    - use of instruction: The use of an instruction is optional. It will be included in the context of the prompt of the agent and can modify the agents behaviour. Please do not include the original text in the prompt. It can be relevant to provide the relatione extraction agent with already extracted entities or entities it should focus on.
    - description: Can extract relations from the text.
    - state access on: text, instruction
    
    URI Detection Agent
    - id: uri_detection_agent
    - use of instruction: The use of an instruction is mandatory. The instruction must be a comma separated list of search terms
    - description: Based on search terms, can determine if there is an associated entity or relation in the Knowledge Graph. The agent will respond with a mapping of search terms to URIs
    - state access on: instruction
     
    Please include in your response exact one agent call using the following agent call structure:
    
    <agent_call>
        <id>AGENT_ID</id>
        <instruction>Put your instructions for the agents here</instruction>
    <agent_call/>
    
    
    
    Agent Call Trace: {call_trace}
    Agent Comments: {comments}
    The provided input text: {text}
    All intermediate results produced during the process: {results}
    
    """)
    
    response_chain = prompt | model
    
    response = response_chain.invoke(state)
    
    agent_id_match = re.search(r'<id>(.*?)</id>', response.content, re.DOTALL)
    if agent_id_match:
        agent_id = agent_id_match.group(1)
    else:
        agent_id = "agent_instructor"
        
    instruction_match = re.search(r'<instruction>(.*?)</instruction>', response.content, re.DOTALL)
    if instruction_match:
        instruction = instruction_match.group(1)
    else:
        instruction = ""
        
    return Command(goto=agent_id, update={"instruction": instruction, "call_trace": state["call_trace"] + [(agent_id, instruction)]})

def entity_extraction_agent(state: cIEState) -> Command[Literal["result_checker_agent"]]:
    prompt = PromptTemplate.from_template("""
    
    You are an expert for entity extraction out of text in a multi-agent-system for closed information extraction. You will receive a text out of the state from which you should extract all entities. In addition, the agent_instructor might give you an instruction, which you should follow. Your task is then to follow the optional instruction as well as this system prompt and return a comma separated list of entities that are in the text, which is enclosed in <result>insert list here</result>. 
    
    The provided input text: {text}
    Instruction: {instruction}
    
    """)
    
    response_chain = prompt | model
    
    response = response_chain.invoke(state)
    
    result_match = re.search(r'<result>(.*?)</result>', response.content, re.DOTALL)
    if result_match:
        result = result_match.group(1)
    else:
        result = ""
    
    result = f"Output of entity_extraction_agent: {result}"
    
    return Command(goto="result_checker_agent", update={"instruction": "", "results": state["results"] + [result]})

def relation_extraction_agent(state: cIEState) -> Command[Literal["result_checker_agent"]]:
    prompt = PromptTemplate.from_template("""
    
    You are an expert for relation extraction out of text in a multi-agent-system for closed information extraction. You will receive a text out of the state from which you should extract all relation. In addition, the agent_instructor might give you an instruction, which you should follow. Your task is then to follow the optional instruction as well as this system prompt and return a list of all triples, where each triple is enclosed in <triple> tags and subject, predicate and object are comma separated from each other. Enclose your pure result in <result> tags
    
    The provided input text: {text}
    Instruction: {instruction}
    
    """)
    
    response_chain = prompt | model
    
    response = response_chain.invoke(state)
    
    result_match = re.search(r'<result>(.*?)</result>', response.content, re.DOTALL)
    if result_match:
        result = result_match.group(1)
    else:
        result = ""
        
    result = f"Output of relation_extraction_agent: {result}"
    
    return Command(goto="result_checker_agent", update={"instruction": "", "results": state["results"] + [result]})

def uri_detection_agent(state: cIEState) -> Command[Literal["result_checker_agent"]]:
    search_terms = state["instruction"].split(",")
    response = ""
    for term in search_terms:
        response += f'Most Similar Detection Results for {term}:{[{"label": doc.page_content, "uri": doc.metadata["uri"]} for doc in vector_store.similarity_search(term, k=3)]}\n\n'
    response = response.replace("},", "},\n")
        
    prompt_template = PromptTemplate.from_template(
        """
        You are a formatting agent. Your task is to check and format the output of the URI detection tool. The tool will give a response like this:
        Most Similar Detection Result for Olaf Scholz: ('label': Angela Merkel, 'uri': 'http://www.wikidata.org/entity/Q567)
        
        Your task is to check the response and output an overall mapping of search terms to URIs. If something doesn't match, please response the non mapping search term with the advise, that those might not be present in the knowledge graph.
        
        URI detection tool response:
        
        {response}
        """
    )
    
    chain = prompt_template | model
    result = chain.invoke({"response": response})
    
    result = f"Output of uri_detection_agent: {result}"
    
    return Command(goto="result_checker_agent", update={"instruction": "", "results": state["results"] + [result]})

def result_checker_agent(state: cIEState) -> Command[Literal["planner", END]]:
    prompt = PromptTemplate.from_template("""
    You are an expert in monitoring multi-agent-systems. In this case you are giving feedback on the process to the planning agent. Therefore, you can see the plans made, as well as agent calls and the history of comments. In addition, you will have access to a text, that should be transformed into triplets, which can be inserted into an underlying knowledge graph. This task often requires multiple iterations to really catch every entity and relation especially those, that are not visible first glimpse. As long as you think the result can be improved, just response with your feedback, which will be processed by the planner in the next step. Really push the result to the edge, what an LLM can do.
    
    In addition to giving feedback, your task is to decide, when the multi-agent-system has come to a reasonable result. If so, just include <FINISH_MAS> in your response. A reasonable result would be, if the result contains just URIs of all relation and all entities and relations can be mapped into the underlying knowledge graph and all triples can be generated. The output will afterwards be formatted by another agent before getting to the user. Please never mention <FINISH_MAS> just in a reasoning response, as it will stop the iteration instantly. 
    
    Agent Call Trace: {call_trace}
    Agent Comments: {comments}
    The provided input text: {text}
    All intermediate results produced during the process: {results}
    """)
    
    response_chain = prompt | model
    
    response = response_chain.invoke(state)
    
    next_agent = "planner"
    
    if "<FINISH_MAS>" in response.content:
        next_agent = END
          
    return Command(goto=next_agent, update={"comments": state["comments"] + [response]})

builder = StateGraph(cIEState)
builder.add_node(planner)
builder.add_node(agent_instructor_agent)
builder.add_node(entity_extraction_agent)
builder.add_node(relation_extraction_agent)
builder.add_node(uri_detection_agent)
builder.add_node(result_checker_agent)

builder.add_edge(START, "planner")

graph = builder.compile()

In [14]:
response_state = graph.invoke({"text": text, "results": [], "call_trace": [], "comments": []}, config={"callbacks": [langfuse_handler]})

In [12]:
response_state

{'text': 'Corfe Castle railway station is a station on the Swanage Railway in the village of Corfe Castle, in the United Kingdom.',
 'call_trace': [('entity_extraction_agent', ''),
  ('entity_extraction_agent',
   'Extract entities from the text and provide a confidence score for each entity. Focus on extracting entities such as "Corfe Castle railway station", "Swanage Railway", "Corfe Castle", and "United Kingdom".'),
  ('relation_extraction_agent',
   'Extract relations among the entities "Corfe Castle railway station", "Swanage Railway", "Corfe Castle", and "United Kingdom" from the text, and provide a confidence score or probability distribution for each extracted relation.')],
 'results': ['Output of entity_extraction_agent: ',
  'Output of entity_extraction_agent: Corfe Castle railway station (1), Swanage Railway (1), Corfe Castle (1), United Kingdom (1)',
  'Output of relation_extraction_agent: \n<triple>Corfe Castle railway station, located on, Swanage Railway, 0.9</triple>\n<t